# Using CordraClient

Using CordraClient to manipulate data on a Cordra digital content management system.

This notebook assumes that you have an instance of the Cordra server running on your computer. The basic test instance of Cordra available for download on the [cordra.org](https://cordra.org) website will work. You will need to specify the `admin` user password in the script to create new users and to upload schemas.

In [1]:
using CordraClient  # Load the library

Let's create a couple new users and a group as the `admin` and then use these users and group in the examples below.

Using this `open(...)`syntax, the connection is automatically closed at the end of the `do` block.

In [10]:
open(CordraConnection, "https://localhost:8443", "admin", "XXXXX", verify=false) do cc
    create_object(cc, Dict("username" => "test", "password" => "thisisatestpassword"), "User", suffix="testuser")
    create_object(cc, Dict("username" => "test2", "password" => "thisisatestpassword"), "User", suffix="test2user")
    create_object(cc, Dict("groupName" => "group1", "description" => "A test group"), "Group", suffix="testGroup")
end

CordraObject(test/testGroup)

Alternatively, we can use this syntax to create a `CordraConnection` which we have to explicitly `close(...)` later.

A `CordraConnection` represents an authorized connection to interact with a Cordra database server.  The method takes a URL, in this case an instance of Cordra running on the same computer.  We specify the username and password we just created above.  Test instances of the Cordra server don't have the necessary certificates so we need to use `verfify=false`.  This is not necessary on full, properly hosted instances of Cordra.

In [64]:
cc = CordraConnection("https://localhost:8443", "test","thisisatestpassword", verify=false)

CordraConnection(https://localhost:8443/test as test)

With a connection we can create digital objects on the Cordra server using the `create_object(...)` method.  These objects will be recorded on the Cordra server and will remain accessible even after disconnecting from and reconnecting to the server or after the server has been shut down and restarted.

A minimalist Cordra object consists of data (created from a `Dict{String, Any}()`) fitting a schema (in this case `Document`.)  We will discuss the data and schema later.

In [3]:
obj = create_object(cc, Dict("name"=>"item 1", "description"=>"This is item 1."), "Document")

CordraObject(test/3655439b62435056f7b4)

The data has been written to the Cordra server and a local copy is returned.

We can examine the properties of the local copy.

In [4]:
content(obj)

Dict{String, Any} with 3 entries:
  "name"        => "item 1"
  "id"          => "test/3655439b62435056f7b4"
  "description" => "This is item 1."

The `content(obj)` is just the data plus an automatically generated `id` which Cordra uses to reference the object on the server.

In [5]:
CordraClient.acl(obj)

Dict{String, Vector{String}} with 2 entries:
  "writers" => ["test"]
  "readers" => ["test"]

By default, the `CordraClient` creates all objects with read and write permissions for the current user.  If we want to create an object which additional users or user groups can access, we add an optional 'acls' argument to the `create_object(...)` method.

In [11]:
obj = create_object(cc, Dict("name"=>"item 2", "description"=>"This is item 2."), "Document", acls = Dict("readers" => [ "test", "test2", "Group"], "writers"=>["test"]))

CordraObject(test/1e3e472767180174848f)

In [12]:
acl(obj)

Dict{String, Vector{String}} with 2 entries:
  "writers" => ["test"]
  "readers" => ["test", "test2", "group1"]

There is also additional `metadata` written with the object.

In [14]:
metadata(obj)

Dict{String, Any} with 5 entries:
  "createdBy"  => "test/testuser"
  "txnId"      => 1657499323418005
  "modifiedBy" => "test/testuser"
  "createdOn"  => 1657499323418
  "modifiedOn" => 1657499323418

In [67]:
using Dates
unix2datetime(metadata(obj)["createdOn"]/1000.0)

2022-07-11T00:28:43.418

We can also ask about the `schema` that the objects data conforms to.  Where by conform, we mean that the data has the necessary fields defined by a JSON schema like those found at [schema.org](https://schema.org/).

`Document` is a very simple schema.  It requires only two fields, "name" and "description".  Our minimalist objects provide these fields and no others. We could have added additional fields with other data items in `Document` or we could use a more sophisticated schema that matchs our digital object.

 [schema.org](https://schema.org/) has many useful schemas and [Bioschemas.org](https://bioschemas.org/index.html) has many biological science-related schemas. You can even create your own schema.

 Regardless of where the schema comes from, you must register them with your instance of Cordra before you use the schema.  Out of the box, Cordra comes with schemas for "User", "Group" and "Document".  We actually used the "User" and "Group" schemas above to create two new Cordra users and a user group.

In [16]:
schema_type(obj)

"Document"

Data properties can be heirarchical, as is demonstrated in this simple example. 

In [18]:
obj = create_object(cc, Dict("name"=>"item3", "description"=>"a little more complex", "properties" => Dict("size"=>10, "color"=>"blue")), "Document")
content(obj)

Dict{String, Any} with 4 entries:
  "name"        => "item3"
  "properties"  => Dict{String, Any}("color"=>"blue", "size"=>10)
  "id"          => "test/9f06640aaa0cb709334f"
  "description" => "a little more complex"

Now we can use these properties to search for objects on the Cordra server instance.  A simple `query(...)` requests those objects with name that starts with `item`.

In [20]:
objs = query(cc, "/name:item*")

4-element Vector{CordraClient.CordraObject}:
 CordraObject(test/496c7d553e1de7908c0d)
 CordraObject(test/3655439b62435056f7b4)
 CordraObject(test/9f06640aaa0cb709334f)
 CordraObject(test/1e3e472767180174848f)

In [22]:
content(objs[3])

Dict{String, Any} with 4 entries:
  "name"        => "item3"
  "properties"  => Dict{String, Any}("color"=>"blue", "size"=>10)
  "id"          => "test/9f06640aaa0cb709334f"
  "description" => "a little more complex"

Alternatively, we could search a little deeper into the object.

In [24]:
objs = query(cc, "/properties/color:blue")
@show objs
content(objs[1])

objs = CordraClient.CordraObject[CordraObject(test/9f06640aaa0cb709334f)]


Dict{String, Any} with 4 entries:
  "name"        => "item3"
  "properties"  => Dict{String, Any}("color"=>"blue", "size"=>10)
  "id"          => "test/9f06640aaa0cb709334f"
  "description" => "a little more complex"

Or even use `AND` and `OR` to join simple search queries into more precise ones.

In [27]:
objs = query(cc, "/properties/color:blue AND /properties/size:10")
@show objs
content(objs[1])

objs = CordraClient.CordraObject[CordraObject(test/9f06640aaa0cb709334f)]


Dict{String, Any} with 4 entries:
  "name"        => "item3"
  "properties"  => Dict{String, Any}("color"=>"blue", "size"=>10)
  "id"          => "test/9f06640aaa0cb709334f"
  "description" => "a little more complex"

Thus it is possible to add digital items to Cordra and retreive them.  We could use options in the `create_object(...)` method to give the object a friendly `id` but realistically, this doesn't scale well as more objects are added.  Best to just let Cordra assign the `id` and use the `query(...)` method to find the objects.

### Payloads
All this is very well and good, however, we often want to store large, arbitrarily formatted items in Cordra.  This is where payloads come in. 

In [32]:
using DataFrames, CSV
df = DataFrame(A=[1,3,5],B=["One","Three","Five"],C=["Uno","Dos","Tres"])
fn = tempname()
CSV.write(fn, df)
obj = create_object(cc, Dict("name"=>"item4","description"=>"item with payload"), "Document", payloads=payload("payload1", fn, "text/csv"))

CordraObject(test/dbb96e411b0d5d5d2847)

Now, if we check the object, we find that it has a payload attached.

In [33]:
payload_names(obj)

1-element Vector{String}:
 "payload1"

We can read the payload back using various mechanisms.  The most basic mechanism returns the payload as a `UInt8[]` which we convert into a string.

In [36]:
String(read_payload(obj,"payload1"))

"A,B,C\n1,One,Uno\n3,Three,Dos\n5,Five,Tres\n"

Export the payload to a file.

In [47]:
fn = export_payload(obj, "payload1", tempname())

"C:\\Users\\nicho\\AppData\\Local\\Temp\\jl_dkadkFIK27"

Process the payload as an IO stream

In [48]:
df = process_payload(obj, "payload1") do io
    CSV.read(io, DataFrame)
end

,A,B,C
,Int64,String7,String7
1,1,One,Uno
2,3,Three,Dos
3,5,Five,Tres


### Schemas 
Schemas define the minimal structure of the data.  Up to this point, we've just used the default, minimal `Document` schema which requires only two entries `name` and `description`.

Schemas are defined in JSON-LD aka JSON for Linked Data.  A good place to find out about schemas is [schema.org](https://schema.org) which defines schemas for many common things.  It would seem to be a good idea to follow the recommendations of schema.org whenever possible but you are welcome to create your own schemas using the syntax and tools provided at the [JSON-LD](https://json-ld.org) site.


Here is an example of a much more complex schema taken from this [example.](https://www.cordra.org/documentation/extensions/person-registry.html)  It builds on the general structure on the [Person page](https://schema.org/Person) at schema.org.
  

```
{
  "type": "object",
  "title": "Person",
  "required": [
    "id",
    "name",
    "birth",
    "gender",
    "address",
    "issuedIds"
  ],
  "properties": {
    "id": {
      "type": "string",
      "cordra": {
        "type": {
          "autoGeneratedField": "handle"
        }
      }
    },
    "name": {
      "type": "object",
      "title": "Name",
      "required": [
        "first",
        "last"
      ],
      "properties": {
        "last": {
          "type": "string",
          "title": "Surname",
          "cordra": {
            "preview": {
              "showInPreview": true,
              "isPrimary": true
            }
          }
        },
        "first": {
          "type": "string",
          "title": "First Name",
          "cordra": {
            "preview": {
              "showInPreview": true
            }
          }
        },
        "middle": {
          "type": "string",
          "title": "Middle Name",
          "cordra": {
            "preview": {
              "showInPreview": true
            }
          }
        }
      }
    },
    "birth": {
      "type": "object",
      "title": "Birth Information",
      "required": [
        "date"
      ],
      "properties": {
        "date": {
          "type": "string",
          "pattern": "^[1-2]{1}[0-9]{7}$",
          "title": "Date of Birth (YYYYMMDD)"
        },
        "certificate": {
          "type": "object",
          "title": "Birth Certificate",
          "required": [
            "id",
            "source"
          ],
          "properties": {
            "id": {
              "type": "string",
              "title": "Certificate ID"
            },
            "source": {
              "type": "string",
              "title": "Certificate Source"
            }
          }
        }
      }
    },
    "death": {
      "type": "object",
      "title": "Death Information",
      "required": [
        "date"
      ],
      "properties": {
        "date": {
          "type": "string",
          "pattern": "^[1-2]{1}[0-9]{7}$",
          "title": "Date of Death (YYYYMMDD)"
        },
        "certificate": {
          "type": "object",
          "title": "Death Certificate",
          "required": [
            "id",
            "source"
          ],
          "properties": {
            "id": {
              "type": "string",
              "title": "Certificate ID"
            },
            "source": {
              "type": "string",
              "title": "Certificate Source"
            }
          }
        }
      }
    },
    "gender": {
      "type": "string",
      "title": "Gender",
      "enum": [
        "female",
        "male",
        "other"
      ]
    },
    "address": {
      "type": "object",
      "title": "Address",
      "required": [
        "line1",
        "line2"
      ],
      "properties": {
        "line1": {
          "type": "string",
          "title": "Line 1"
        },
        "line2": {
          "type": "string",
          "title": "Line 2"
        },
        "line3": {
          "type": "string",
          "title": "Line 3"
        }
      }
    },
    "issuedIds": {
      "type": "array",
      "title": "Government Issued Ids",
      "format": "table",
      "uniqueItems": true,
      "minItems": 1,
      "items": {
        "type": "object",
        "required": [
          "type",
          "id"
        ],
        "properties": {
          "id": {
            "type": "string",
            "title": "ID"
          },
          "type": {
            "type": "string",
            "title": "ID Type"
          }
        }
      }
    },
    "fingerprints": {
      "type": "object",
      "title": "Finger Print External Reference",
      "required": [
        "id",
        "source",
        "lastCapturedDate"
      ],
      "properties": {
        "id": {
          "type": "string",
          "title": "Certificate ID"
        },
        "source": {
          "type": "string",
          "title": "Certificate Source"
        },
        "lastCapturedDate": {
          "type": "string",
          "pattern": "^[1-2]{1}[0-9]{7}$",
          "title": "Last Captured Date (YYYYMMDD)"
        }
      }
    },
    "recordCreatedOn": {
      "type": "string",
      "title": "Record Creation Date",
      "cordra": {
        "type": {
          "autoGeneratedField": "creationDate"
        }
      }
    },
    "recordModifiedOn": {
      "type": "string",
      "title": "Record Modification Date",
      "cordra": {
        "type": {
          "autoGeneratedField": "modificationDate"
        }
      }
    },
    "recordCreatedBy": {
      "type": "string",
      "title": "Record Created By",
      "cordra": {
        "type": {
          "autoGeneratedField": "createdBy"
        }
      }
    },
    "recordModifiedBy": {
      "type": "string",
      "title": "Record Modified By",
      "cordra": {
        "type": {
          "autoGeneratedField": "modifiedBy"
        }
      }
    }
  }
}
```

Let's create a simple schema for a generic `Sample`-type.

In [59]:
sample_schema = """
{
  "type": "object",
  "title": "Sample",
  "required": [
    "name",
    "description",
    "preparation",
    "prepared_by"
  ],
  "properties": {
    "id": {
      "type": "string",
      "cordra": {
        "type": {
          "autoGeneratedField": "handle"
        }
      }
    },
    "name": {
      "type": "string",
      "title": "Name"
    },
    "description": {
      "type": "string",
      "title": "Description"
    },
    "preparation": {
      "type": "string",
      "title": "Preperation"
    },
    "prepared_by": {
      "type": "string",
      "title": "Prepared by"
    }
  }
}
"""

"{\n  \"type\": \"object\",\n  \"title\": \"Sample\",\n  \"required\": [\n    \"name\",\n    \"description\",\n    \"preparation\",\n    \"prepared_by\"\n  ],\n  \"properties\": {\n    \"id\": {\n      \"type\": \"string\",\n      \"cordra\": {\n        \"type\": {\n          \"autoGeneratedField\": \"handle\"\n        " ⋯ 61 bytes ⋯ "\"title\": \"Name\"\n    },\n    \"description\": {\n      \"type\": \"string\",\n      \"title\": \"Description\"\n    },\n    \"preparation\": {\n      \"type\": \"string\",\n      \"title\": \"Preperation\"\n    },\n    \"prepared_by\": {\n      \"type\": \"string\",\n      \"title\": \"Prepared by\"\n    }\n  }\n}\n"

You must be an `admin` to create a Schema.

In [58]:
open(CordraConnection, "https://localhost:8443", "admin","XXXXX",verify=false) do cc2
    create_schema(cc2, "Sample", JSON.parse(sample_schema))
end

true

Let's first create a valid sample.

In [60]:
samp = create_object(cc, Dict("name"=>"fish","description"=>"A fish","preparation"=>"sauted","prepared_by"=>"The Swedish chief."),"Sample")

CordraObject(test/381fa72e2d2d99a316dc)

Then an invalid one.

In [62]:
create_object(cc, Dict("name"=>"fowl","description"=>"A bird","preparation"=>"boiled"),"Sample")

ErrorException: 400 Bad Request. : object has missing required properties (["prepared_by"])

It errors helpfully telling us that it is missing a required property `prepared_by.`  Thus schemas can be used to ensure that all the necessary data is associated with an object.

Finally, we close the connection to the Cordra server.

In [63]:
close(cc)

true